# I: Literature Reading

> The purpose of this part of the project is to understand the concept and the skills on Merkle tree, hash collision, and hash puzzle. It also practices literature review skills. In the end, you need to list references that cover these concepts and their latest development. The minimum number of the reference list is three. It is no harm to list more. Along with each reference, you need give a paragraph briefing the main ideas. Note, reference here is meant for any materials that help you understand the concepts and progress the projects. They can be a book chapter, a peer reviewed journal or conference paper or a source from the internet.

I list the following three references that can be your starting points. All of them are posted for your
convenience. You cannot use them as your references.

R1: B. Weber and X. Zhang. Parallel hash collision search by rho method with distinguished points. Proc.
of the 14th IEEE LISAT 2018, Farmingdale, NY, May 4, 2018, pp. 1-7

R2: Mark Stamp book (listed in the course syllabus), section 5.2 (Birthday attack, Nostradamus attack)
and 5.3 (MD4).

R3: J. Kelsey and T. Kohno, Herding hash functions and the Nostradamus attack,
eprint.iacr.org/2005/28l.pdf; It is posted in the assignment folder.



# II: Questions

## Q1
> Why is hash collision inevitable mathematically?

## Q2 
> Suppose that Trudy is in a room containing a total of N people (including herself). What is the probability that at least one of the other N - 1 people have the same birthday as Trudy? What is the minimum N such that the probability is over 50%?

## Q3 
> What is the probability that any two (or more) people in a room share the same birthday, where there are N people (N<=365) in the room? What is the minimum number of N such that the chance is over 50%?

## Q4
> What is the main idea behind the birthday attack on hash? How can a birthday attack improve the efficiency of an attack compared to a naive brute force attack?

## Q5 
> What is the main potential issue for hash function that is constructed using Merkle-Damagurd Construction process?


## Q6 
> (optional) Open discussion on the main theme of the project: can we predict the future?

# III: Code Project 

## 1. Merkel tree implementation
 
### a.

> Use SHA256 or any other hash algorithms in Merkle trees. You do not need to implement hash algorithms unless you want to. Explore crypto packages such as pycryptodome. Most of them shall have the implementation


In [4]:
from hashlib import sha256
from typing import List, Optional
from IPython.display import Markdown
from project6.mermaid import Mermaid

class Node:
    left: Optional["Node"] = None
    right: Optional["Node"] = None
    parent: Optional["Node"] = None
    _value: Optional[str] = None

    def __init__(self, value: Optional[str] = None):
        self._value = value

    @property
    def digest(self) -> bytes:
        m = sha256()
        if self.left is not None and self.right is not None:
            m.update(self.left.digest)
            m.update(self.right.digest)
        else:
            assert self._value is not None
            m.update(self._value.encode())

        return m.digest()

    @property
    def value(self):
        return self.digest.hex()[:8]


class Tree:
    @staticmethod
    def build(nodes: List[Node]):
        n = len(nodes)
        for i in range(n):
            if i * 2 + 1 < n:
                nodes[i].left = nodes[i * 2 + 1]
                nodes[i * 2 + 1].parent = nodes[i]
            if i * 2 + 2 < n:
                nodes[i].right = nodes[i * 2 + 2]
                nodes[i * 2 + 2].parent = nodes[i]
        return nodes[0]


### b.
> Each leaf node points to a plain text file. The text file can have any contents.

NOTE: We opted for just using strings instead of files to make the code simpler.

In [ ]:
four_leaves = [
    Node(),
    Node(),
    Node(),
    Node('A'),
    Node('B'),
    Node('C'),
    Node('D'),
]
six_leaves = [
    Node(),
    Node(),
    Node(),
    Node(),
    Node(),
    Node('C'),
    Node('D'),
    Node('A'),
    Node('B'),
    Node('E'),
    Node('F'),
]

root_four_leaves = Tree.build(four_leaves)
root_six_leaves = Tree.build(six_leaves)

In [ ]:
Markdown(f"""
### c.
         
> Test case 1 with four leaf nodes. Print out the tree structure and the corresponding hashes

![root_four_leaves]({Mermaid.render_binary_search_tree(root_four_leaves).to_url()})
""")

In [ ]:

Markdown(f"""
### d.

> Test case 2 with six leaf nodes. Print out the tree structure and the corresponding hashes.
         
![root_six_leaves]({Mermaid.render_binary_search_tree(root_six_leaves).to_url()})
""")

## 2. Root hash Observation
> Under four leaf node case, alter one of the text file that is pointed by one of the leaf node. Alter here means you change the context of the file. Compare the root hash with the one with original root hash. What can you say about and why?

Any changes to any of the node's value, will result in any parent nodes, as well as the root node, being changed as well. This is because all children hash changes bubble-up, as a node hash is the resulting hash of the concatention of both of it's children's hashes. In the case of no children, the node's value itself is hashed, wherein any changes to said value will change the hash, which then propagates upwards to all nodes of the same subtree.

In [ ]:
four_leaves = [
    Node(),
    Node(),
    Node(),
    Node('Z'),
    Node('B'),
    Node('C'),
    Node('D'),
]

root_four_leaves = Tree.build(four_leaves)

In [ ]:
Markdown(f"""
### 4 Leaf Nodes with change

`A` has been changed to `Z`. Root hash is now `81205ce6` instead of `1b3faa3f`.

![root_four_leaves]({Mermaid.render_binary_search_tree(root_four_leaves).to_url()})
""")

## 3. Hash Collision


### a.

>Take any one text file that is attached to one leaf node, Generate as many text files as possible with the same meaning. For example, adding more spaces to the file that does not change the meaning of the file but should generate different hash. You may try 8 files first. Test your luck if you can find a hash collision.


In [10]:
initial_val = 'INITIALTEXTFILE'
one_leaf = [
    Node(),
    Node(initial_val),
]
root_one_leaf = Tree.build(one_leaf)
original_val = root_one_leaf.left.value

# will just hash changed values directly as they will show the same values
full_initial_val = sha256(initial_val.encode()).hexdigest()[:8]
assert(original_val == full_initial_val[:8])

# add variable amount of spaces
with_one_space = sha256((initial_val + ' ').encode()).hexdigest()[:8]
assert(with_one_space != full_initial_val)
with_two_spaces = sha256((initial_val + (' ' * 2)).encode()).hexdigest()[:8]
assert(with_one_space != with_two_spaces) #even adding different amounts of spaces changes the hash value

# with tabs 
with_one_tab = sha256((initial_val + '\t').encode()).hexdigest()[:8]
assert(with_one_tab != full_initial_val)
with_two_tabs = sha256((initial_val + ('\t' * 2)).encode()).hexdigest()[:8]
assert(with_one_tab != with_two_tabs) #even adding different amounts of tabs changes the hash value

# with new lines
with_new_line = sha256((initial_val + '\n').encode()).hexdigest()[:8]
assert(with_new_line != full_initial_val)
with_two_new_lines = sha256((initial_val + ('\n' * 2)).encode()).hexdigest()[:8]
assert(with_new_line != with_two_new_lines) #even adding different amounts of new lines changes the hash value

# with null
with_null = sha256(initial_val.encode() + b'\x00').hexdigest()[:8]
assert(with_null != full_initial_val)
with_two_nulls = sha256(initial_val.encode() + (b'\x00' * 2)).hexdigest()[:8]
assert(with_null != with_two_nulls) #even adding different amounts of nulls (invisible control characters) changes the hash value


### b.

>In general, with hash256, it is rarely luck to find a collision. Even changing the hash to a weak one, say MD4, you still could not find hash collision unless you are using specific techniques like rho method or herding. Again, no collision found is expected. For the class project, we can reduce the hash output length to 4-bit hash. You know how many files you need to generate so as to find collision with naïve exhaustive attack. Demo the result.


In [12]:
from Crypto.Hash import MD4

# Test with MD4

# will just hash changed values directly as they will show the same values
full_initial_val_md4 = MD4.new(initial_val.encode()).hexdigest()[:8]

# add variable amount of spaces
with_one_space_md4 = MD4.new((initial_val + ' ').encode()).hexdigest()[:8]
assert(with_one_space_md4 != full_initial_val_md4)
with_two_spaces_md4 = MD4.new((initial_val + (' ' * 2)).encode()).hexdigest()[:8]
assert(with_one_space_md4 != with_two_spaces_md4) #even adding different amounts of spaces changes the hash value

# with tabs 
with_one_tab_md4 = MD4.new((initial_val + '\t').encode()).hexdigest()[:8]
assert(with_one_tab_md4 != full_initial_val_md4)
with_two_tabs_md4 = MD4.new((initial_val + ('\t' * 2)).encode()).hexdigest()[:8]
assert(with_one_tab_md4 != with_two_tabs_md4) #even adding different amounts of tabs changes the hash value

# with new lines
with_new_line_md4 = MD4.new((initial_val + '\n').encode()).hexdigest()[:8]
assert(with_new_line_md4 != full_initial_val_md4)
with_two_new_lines_md4 = MD4.new((initial_val + ('\n' * 2)).encode()).hexdigest()[:8]
assert(with_new_line_md4 != with_two_new_lines_md4) #even adding different amounts of new lines changes the hash value

# with null
with_null_md4 = MD4.new(initial_val.encode() + b'\x00').hexdigest()[:8]
assert(with_null_md4 != full_initial_val_md4)
with_two_nulls_md4 = MD4.new(initial_val.encode() + (b'\x00' * 2)).hexdigest()[:8]
assert(with_null_md4 != with_two_nulls_md4) #even adding different amounts of nulls (invisible control characters) changes the hash value


In [27]:
# Brute force 4 digit hash
import random
import string

initial_md4_4_digit = full_initial_val_md4[:4]
for rand_iteration in range(65536):
    text = ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))
    this_round_hash = MD4.new(text.encode()).hexdigest()[:4]
    if this_round_hash == initial_md4_4_digit:
        print("found collision:", this_round_hash, initial_md4_4_digit, text)
        break


found collision: a303 a303 XSIUD


### c.
> Short discuss if you have 8-bit hash all the way to 160-bit hash what your strategy would be to get possible collision

## 4. Hash Puzzle

### a.

> Stick with the hash algorithm you used in 1), practice so called hash puzzle. It is basicallyfinding certain hash that satisfies a set condition. For example, the first 8 bits (prefix-) must be zero. Once you find two hashes having the same prefix zeros, you can say you find the collision. In this project, you start with puzzle having 1-bit zero in the beginning, increase to 4-bit zero. Demo the result and explain for the worst cases how many hashes you need to generate.

### b.

> Reason briefly the workload if we want to solve a puzzle with enough zeros in front say 20